In [95]:
import pandas as pd
from pandas import DataFrame

import json
import re
import glob

import sqlite3

import warnings
warnings.filterwarnings('ignore')

In [96]:
def read_json(filepath: str) -> dict:
    """Read json from file."""
    with open(filepath) as f:
        data = json.load(f)
    return data

def extract_filename(string: str) -> str:
    """Return filename in titlecase."""
    pattern = r'\/([^\/]+)\.json$'
    match = re.search(pattern, string)
    if match:
        file_name = match.group(1)
        return file_name.replace('-', ' ').title()
    
def load_dfs(dir_path: str, category: str) -> DataFrame:
    """Return list of dataframes."""
    dataframes = []
    files = glob.glob(dir_path)

    for file in files:
        data = read_json(file)
        df = pd.json_normalize(data['data']['rolesByAliases'])  # Break down nested dictionary.
        df[category] = extract_filename(file)
        dataframes.append(df)
    
    return pd.concat(dataframes)

def create_surrogate_keys(sk_map: dict, df: DataFrame):
    """"""
    for col, sk in sk_map.items():
        df[sk] = pd.factorize(df[col])[0] + 1

In [97]:
# Load dataframes.
industries = load_dfs(dir_path='../data/industries/*.json', category='industry')
themes = load_dfs(dir_path='../data/themes/*.json', category='theme')

# Create master df.
df = pd.concat([industries, themes])
df.shape

(3286, 30)

In [98]:
# Drop unused columns
df.drop(columns=['roleReviewStatistics', 'growth', 'salarySuggestion'],
        inplace=True)

# Rename columns
col_names = {
    'specReqs': 'skill',
    'roleReviewStatistics.total': 'totalReviews',
    'roleReviewStatistics.remuneration': 'remuneration',
    'roleReviewStatistics.employability': 'employability',
    'roleReviewStatistics.jobSatisfaction': 'jobSatisfaction',
    'roleReviewStatistics.workLifeBalance': 'workLifeBalance',
    'roleReviewStatistics.diversityInTasks': 'diversityInTasks',
    'roleReviewStatistics.careerProgressionOpportunities': 'careerProgressionOpportunities',
    'roleReviewStatistics.overtime.never': 'overtime.never',
    'roleReviewStatistics.overtime.often': 'overtime.often',
    'roleReviewStatistics.overtime.sometimes': 'overtime.sometimes',
    'roleReviewStatistics.weekends.never': 'weekends.never',
    'roleReviewStatistics.weekends.often': 'weekends.often',
    'roleReviewStatistics.weekends.sometimes': 'weekends.sometimes',
    'roleReviewStatistics.shiftWork.never': 'shiftWork.never',
    'roleReviewStatistics.shiftWork.often': 'shiftWork.often',
    'roleReviewStatistics.shiftWork.sometimes': 'shiftWork.sometimes',
    'roleReviewStatistics.lateNights.never': 'lateNights.never',
    'roleReviewStatistics.lateNights.often': 'lateNights.often',
    'roleReviewStatistics.lateNights.sometimes': 'lateNights.sometimes',
    'roleReviewStatistics.workingHoursCount': 'workingHoursCount',
    'growth.value': 'growth',
    'salarySuggestion.salary_median': 'medianSalary',
}

df.rename(columns=col_names, inplace=True)

# Update dtypes
cols = ['remuneration', 'employability', 'jobSatisfaction',
        'workLifeBalance', 'diversityInTasks', 'careerProgressionOpportunities']

df = df.astype({col: float for col in cols})

In [99]:
# Break down skills column.
df = df.explode('skill')
# Parse the json structure to get the skill.
df['skill'] = df['skill'].apply(lambda x: x.get('label') if x is not None else None)

In [100]:
# Create surrogate (primary) keys
keys = {
    'theme': 'themeId',
    'industry': 'industryId',
    'id': 'jobId',
    'skill': 'skillId'
}

create_surrogate_keys(sk_map=keys, df=df)

# Normalisation

In [101]:
# Themes
themes = df[['themeId', 'theme']]
themes.dropna(inplace=True)
themes.drop_duplicates(inplace=True)
themes.set_index('themeId', inplace=True)

# Industries
industries = df[['industryId', 'industry']]
industries.dropna(inplace=True)
industries.drop_duplicates(inplace=True)
industries.set_index('industryId', inplace=True)

# Jobs
jobs = df.drop(columns=['industry', 'industryId', 'theme', 'themeId', 'skill', 'skillId'])
jobs.drop_duplicates(inplace=True)
jobs.set_index('jobId', inplace=True)

# Skills
skills = df[['skillId', 'skill']]
skills.dropna(inplace=True)
skills.drop_duplicates(inplace=True)
skills.set_index('skillId', inplace=True)

# JobsThemes
jobs_themes = df[['jobId', 'themeId']]
jobs_themes = jobs_themes[jobs_themes['themeId'] > 0]
jobs_themes.drop_duplicates(inplace=True)

# JobsIndustries
jobs_industries = df[['jobId', 'industryId']]
jobs_industries = jobs_industries[jobs_industries['industryId'] > 0]
jobs_industries.drop_duplicates(inplace=True)

# JobsSkills
jobs_skills = df[['jobId', 'skillId']]

# Load

Load dataframes into sqlite3 file.

In [102]:
connection = sqlite3.connect('seek.db')

jobs.to_sql('Jobs', connection, index=True, if_exists='replace')
industries.to_sql('Industries', connection, index=True, if_exists='replace')
themes.to_sql('Themes', connection, index=True, if_exists='replace')
skills.to_sql('Skills', connection, index=True, if_exists='replace')

jobs_industries.to_sql('JobsIndustries', connection, index=False, if_exists='replace')
jobs_themes.to_sql('JobsThemes', connection, index=False, if_exists='replace')
jobs_skills.to_sql('JobsSkills', connection, index=False, if_exists='replace')

connection.close()